In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批处理维度的其他所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1053, -0.0072,  0.0952,  0.0777, -0.0229, -0.0145,  0.1509, -0.0424,
         -0.0265,  0.0367]], grad_fn=<AddmmBackward>)


In [4]:
output = net(input)
target = torch.randn(10)  # 本例子中使用模拟数据
target = target.view(1, -1)  # 使目标值与数据值形状一致
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8800, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()     # 清零所有参数(parameter）的梯度缓存

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0020, -0.0001, -0.0028,  0.0047,  0.0136,  0.0105])


In [9]:
import  torch
x = torch.ones(2, 4, requires_grad=True)
print(x)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)


In [10]:
y = x + 2
print(y)
print(y.requires_grad)

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.]], grad_fn=<AddBackward0>)
True


In [13]:
x.requires_grad_(False)

print(x.requires_grad, y.requires_grad)

False True


In [18]:
#要使得x支持求导，必须x为浮点类型
x = torch.tensor([[1., 2., 3.],[4., 5., 6.]], requires_grad=True)

y = x + 1
z = 2 * y * y
j = torch.mean(z)

In [19]:
#求导只能是标量对标量 或者是标量对矩阵或者向量， 反之不行
z.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [20]:
j.backward()

In [21]:
x.grad

tensor([[1.3333, 2.0000, 2.6667],
        [3.3333, 4.0000, 4.6667]])